In [304]:
#Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import ast 
import scipy.stats as stats
import math
import seaborn as sns
from scipy.optimize import curve_fit
from ipywidgets import interact
from scipy.stats import pearsonr

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import statsmodels.api as sm
from sympy import symbols, integrate, lambdify, diff
import scipy.integrate as spi
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import f_oneway, kruskal
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture


from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import DBSCAN


### Define Dataset to conduct analysis on: 

In [305]:
time_between_frames = 5
initial_exposure = 60
time_between_exposure = 120
num_frames = 289

framerange = [(0, 25), (25, 50), (50, 75), (75, 100), (100, 300)]
sample_germination_frames = [14, 38, 62, 86]
columns_to_scale = ['INTENSITY', 'AREA', 'GERMINANT EXPOSURE', 'GERMINATION',
                    'ELLIPSE MINOR', 'ELLIPSE MAJOR', 'PERIMETER', 'CIRCULARITY', "ELLIPSE ASPECT RATIO"]


In [306]:
experiment = "combined"
#experiment = "M4581_s1"
#experiment = "M4567_s2"

---
# Analysis 

### Initialization: includes calculating exposure, bin width, and normalization functions

In [307]:
def calculate_exposure(num_frames, initial_exposure, time_between_exposure, time_between_frames) -> list[int]:

  frames_between_exposures = time_between_exposure/time_between_frames
  first_frame_exposure = initial_exposure/time_between_frames - 1

  print(f"first germinant exposure: {first_frame_exposure}")
  print(f"time between germinant exposures: {frames_between_exposures}")

  exposure_frames = [first_frame_exposure]

  while exposure_frames[-1] + frames_between_exposures < num_frames:
    exposure_frames.append(exposure_frames[-1] + frames_between_exposures)

  return first_frame_exposure, frames_between_exposures, exposure_frames

# function call 
first_frame_exposure, frames_between_exposures, germinant_exposures = calculate_exposure(num_frames, initial_exposure, time_between_exposure, time_between_frames)
print(f"Germinant exposures: {germinant_exposures}")

first germinant exposure: 11.0
time between germinant exposures: 24.0
Germinant exposures: [11.0, 35.0, 59.0, 83.0, 107.0, 131.0, 155.0, 179.0, 203.0, 227.0, 251.0, 275.0]


Bin Width using (1) Sturges Method (use with small datasets) (2) Scott's Rule normally distributed data (3) Freedman-Diaconis Rule - robust to outliers

In [308]:
def get_bin_width(data, method):  
  if method == "Sturges":
    k = int(np.ceil(np.log2(len(data)) + 1))
    bin_width = (np.max(data) - np.min(data)) / k

  if method == "Scotts":
    bin_width = 3.5 * np.std(data) / (len(data) ** (1/3))

  if method == "Freedman":
    q25, q75 = np.percentile(data, [25, 75])
    iqr = q75 - q25
    bin_width = 2 * iqr / (len(data) ** (1/3))

  bins = int(np.ceil((data.max() - data.min()) / bin_width))
  return bins

Normalize DataSet: 

In [309]:
def scale_csv_data(file_path: str, columns_to_scale: list[str]):
    data = pd.read_csv(file_path)
    scaler = MinMaxScaler()

    def scale_column(column):
        lists = column.apply(ast.literal_eval)
        flattened = np.concatenate(lists.values)
        scaled_flattened = scaler.fit_transform(flattened.reshape(-1, 1)).flatten()
        scaled_lists = []
        start = 0
        for lst in lists:
            end = start + len(lst)
            scaled_lists.append(scaled_flattened[start:end].tolist())
            start = end
        return [str(lst) for lst in scaled_lists]
    
    for column in columns_to_scale:
        data[column] = scale_column(data[column])

    return data

Datasets:

In [310]:
#M4581_s1
csv_path = "/Users/alexandranava/Desktop/Spores/M4581_s1/Analysis/V3/M4581_s1_Model_Data.csv"
plots_folder_M4581s1 = "/Users/alexandranava/Desktop/Spores/M4581_s1/Analysis/V3/Plots/"
M4581_s1_df  = scale_csv_data(csv_path, columns_to_scale)

#M4576_s2
csv_path2 = "/Users/alexandranava/Desktop/Spores/M4576_s2/M4576_s2_Model_Data.csv"
plots_folder_M4576s2 = "/Users/alexandranava/Desktop/Spores/M4576_s2/Analysis/Plots/"
M4576_s2_df  = scale_csv_data(csv_path2, columns_to_scale)

#combined datasets
plots_folder_combined = "/Users/alexandranava/Desktop/Spores/M4581_s1andM4576_s2/plots/"
M4581_s1_scaleddf  = scale_csv_data(csv_path, columns_to_scale)
M4576_s2_scaleddf = scale_csv_data(csv_path2, columns_to_scale)
df_stacked = pd.concat([M4581_s1_scaleddf, M4576_s2_scaleddf], axis=0)
df_stacked.reset_index(drop=True, inplace=True)

Set dataframe to conduct analysis on:

In [311]:
if experiment == "combined": 
  df = df_stacked
  combined_dataset = 1
  plots_folder = plots_folder_combined
if experiment == "M4567_s2":
  df = M4576_s2_scaleddf
  combined_dataset = 0
  plots_folder = plots_folder_M4576s2
if experiment == "M4581_s1":
  df = M4581_s1_scaleddf
  combined_dataset = 0
  plots_folder = plots_folder_M4581s1

## Germination Measurement

Adds Germination Frame to df in "GERMINATION_INDEX" column

In [312]:
def add_germination_index(df):
    GERMINATION: list[list] = df["GERMINATION"].apply(ast.literal_eval)
    germination_frames_list = GERMINATION.apply(lambda x: x.index(1))

    df["GERMINATION_INDEX"] = germination_frames_list

add_germination_index(df)

Adds germinant exposures before germination to df in "GERMINANT_EXPOSURES" column

In [313]:
def add_germinant_exposures(df):
  EXPOSURE_COLUMN = [] # {exposure number: spores germinated with that exposure}

  for frame_number in df["GERMINATION_INDEX"]:
    exposure_count = math.floor((frame_number - first_frame_exposure)/frames_between_exposures) + 1
    EXPOSURE_COLUMN.append(exposure_count)
  df["GERMINANT_EXPOSURES"] = EXPOSURE_COLUMN

add_germinant_exposures(df)

Creates a lineplot of population dormancy, as well as a histogram of germination events

In [314]:
def germination_frequency(plots_folder: str, df, output: int = 1) -> None:

  germination_frames_list = list(df["GERMINATION_INDEX"])
  frames_shown: int = 189


  GERMINATION = []
  min_frames = 10000
  for row in df["GERMINATION"].apply(ast.literal_eval):
    if len(row)<min_frames:
      min_frames = len(row)

  for row in df["GERMINATION"].apply(ast.literal_eval):
    GERMINATION.append(row[:min_frames])

  bin_size = get_bin_width(np.array(GERMINATION), "Sturges")
  print(f"optimal bin width: {bin_size}")

  #print times frame germination occurs
  frame_counts = Counter(germination_frames_list)
  sorted_frame_counts = sorted(frame_counts.items())

  germination_events: list[int] = []
  percent_germinated_at_t: list[int] = []
  for frame_number, count in sorted_frame_counts:
      if output == 1:
        germination_events.append(count)
  #histogram plot 
  fig, ax1 = plt.subplots()

  #germinant intervals
  ax1.grid(color='silver', linewidth=1, linestyle = "dashed", which = "both", axis = "x")

  #histogram
  sns.histplot(germination_frames_list, bins = bin_size, label = f"Germination Events", ax = ax1, color = "powderblue")
  
  #axis labels 
  ax1.set_xlabel("Frame", fontsize = 16)
  ax1.set_ylabel("Germination Event Frequency", fontsize = 16)

  #x axis limit and ticks
  ax1.set_xlim([0, frames_shown])
 
  #overlaid percentage plot 
  total_spores = sum(count for frame, count in sorted_frame_counts)
  spores_count = total_spores
  frames = [0]
  percents = [100]
  percent_plot = [100]
  frame_dict = dict(sorted_frame_counts)

  total_percent = 100  

  ### by individual frames
  for frame_number in range(1, frames_shown+1):
    if frame_number in frame_dict:
      count = frame_dict[frame_number]
      spores_count -= count
      total_percent = spores_count / total_spores * 100
      t_percent = count/total_spores * 100
      percent_plot.append(total_percent)
      percent_germinated_at_t.append(t_percent)
    percents.append(total_percent)
    frames.append(frame_number)

  #percent plot
  ax2 = ax1.twinx()
  sns.lineplot(x=frames, y=percents, ax = ax2, linestyle = "--", linewidth = 2, label = "Dormant Percentage", color = "tomato")

  #custom germinant label
  ax2.axvline(x=11, color='silver', label='Germinant Exposure', linewidth = 1, linestyle = "dashed")

  #percent y axis formatting
  ax2.set_ylabel("Dormant Percentage", fontsize = 16)
  ax2.set_yticks([0, 25, 50, 75, 100])
  ax2.tick_params(axis='y', labelsize=12)
  ax2.yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{int(y)}%'))
  
  #legends
  handles1, labels1 = ax1.get_legend_handles_labels()
  handles2, labels2 = ax2.get_legend_handles_labels()
  ax2.legend(handles=handles1 + handles2, labels=labels1 + labels2, loc='best', fontsize=12)

  plt.tight_layout()
  plt.savefig(f"{plots_folder}germination_percentage_{bin_size}bins.jpg")
  plt.clf()
  
  ### Stats
  print(f"Total Spores: {total_spores}")

  ## statistics by frames
  print("\n Statistics by Events:")
  print(f"Frames that events occur: {len(germination_events)}")
  print(f"Frames: {str(list(frame_dict.keys())).replace(',', ' &')}")
  print(f"Germination Events: {str(germination_events).replace(',', ' &')}")
  print(f"Total Percentage: [{' & '.join([f'{elem:.0f}' for elem in percent_plot[1:]])}]")
  print(f"Percentage at frame: [{' & '.join([f'{elem:.0f}' for elem in percent_germinated_at_t[1:]])}]")

germination_frequency(plots_folder, df)

optimal bin width: 9
Total Spores: 130

 Statistics by Events:
Frames that events occur: 15
Frames: [13 & 14 & 16 & 37 & 38 & 39 & 40 & 57 & 61 & 62 & 63 & 64 & 85 & 86 & 182]
Germination Events: [5 & 5 & 1 & 24 & 42 & 2 & 1 & 3 & 15 & 21 & 2 & 1 & 1 & 6 & 1]
Total Percentage: [96 & 92 & 92 & 73 & 41 & 39 & 38 & 36 & 25 & 8 & 7 & 6 & 5 & 1 & 0]
Percentage at frame: [4 & 1 & 18 & 32 & 2 & 1 & 2 & 12 & 16 & 2 & 1 & 1 & 5 & 1]


<Figure size 640x480 with 0 Axes>

Creates a piechart of germinant exposures until germination

In [315]:
def piechart_germination_exposure(df):
    total_spores = len(df)
    df = df.sort_values(by=['GERMINANT_EXPOSURES'])
    exposures_list = df['GERMINANT_EXPOSURES'].unique()
    value_counts = df['GERMINANT_EXPOSURES'].value_counts()

    #========================GERMINANT EXPOSURE
    ## statistics by germinant exposures
    total_percentage_by_exposure = []
    percentage_at_exposure = []

    total_germinated = 0
    for exposure in exposures_list:
        spores_germinated_interval = value_counts[exposure]
        total_germinated += spores_germinated_interval
        total_percentage_by_exposure.append(((total_spores - total_germinated) / total_spores) * 100)

        percentage_at_t = (spores_germinated_interval / total_spores) * 100
        percentage_at_exposure.append(percentage_at_t)

    ### Pie chart by germinant exposures
    labels = [f'$GE_{exposure}$ = {value_counts[exposure] / total_spores * 100 :.2f}%' for exposure in exposures_list]

    plt.clf()
    colors = sns.color_palette('pastel')[0:len(percentage_at_exposure)]

    explode_val = percentage_at_exposure.index(max(percentage_at_exposure))
    explode = [0 for _ in range(len(percentage_at_exposure))]
    explode[explode_val] = 0.1

    plt.pie(percentage_at_exposure, labels=labels, shadow=True, colors=colors, wedgeprops={'edgecolor': 'black'}, startangle=140, explode=explode, pctdistance=1.00)
    plt.title("Distribution of Germination Exposures")
    plt.savefig(plots_folder + "piechart_germination_atexposure.jpg")
    plt.clf()

    ## Print stats
    print("\n Statistics by Exposures:")
    print(f"Germinant Exposures: {str(list(exposures_list)).replace(',', ' &')}")
    print(f"Germination Events: {str(list(value_counts)).replace(',', ' &')}")
    print(f"Total Dormancy Percentage: [{' & '.join([f'{elem:.2f}' for elem in total_percentage_by_exposure])}]")
    print(f"Percent Germinated at Exposure: [{' & '.join([f'{elem:.2f}' for elem in percentage_at_exposure])}]")

piechart_germination_exposure(df)


 Statistics by Exposures:
Germinant Exposures: [1 & 2 & 3 & 4 & 8]
Germination Events: [72 & 39 & 11 & 7 & 1]
Total Dormancy Percentage: [91.54 & 36.15 & 6.15 & 0.77 & 0.00]
Percent Germinated at Exposure: [8.46 & 55.38 & 30.00 & 5.38 & 0.77]


<Figure size 640x480 with 0 Axes>

## Feature Analysis

Get minimum and maximum global value of a given feature

In [316]:
def get_min_max(df, feature: str) -> int:
    data_max = []
    data_min = []
    data = df[feature].apply(ast.literal_eval) 
    for row in data:
        data_max.append(max(row))
        data_min.append(min(row))
    return [min(data_min), max(data_max)]

minmax_intensity: list[int, int] = get_min_max(df, "INTENSITY")
minmax_area: list[int, int] = get_min_max(df, "AREA")
minmax_minor: list[int, int] = get_min_max(df, "ELLIPSE MINOR")
minmax_major: list[int, int] = get_min_max(df, "ELLIPSE MAJOR")
minmax_perimeter: list[int, int] = get_min_max(df, "PERIMETER")
minmax_circularity: list[int, int] = get_min_max(df, "CIRCULARITY")
minmax_aspectratio: list[int, int] = get_min_max(df, "ELLIPSE ASPECT RATIO")

Adds initial feature value, last feature value before germination, and the difference between these two in "INITIAL_{feature}", "LAST_{feature}" and "DIFF_{feature}" columns

In [317]:
def add_initial_last_measure_feature(df, feature:str):

  FEATURE = df[feature].apply(ast.literal_eval)

  INITIAL_FEATURE_COLUMN = []
  LAST_FEATURE_COLUMN = []
  DIFF_FEATURE_COLUMN = []

  for i in range(len(df)):
    spore_feature = FEATURE[i]
    spore_germination_frame = df.loc[i, "GERMINATION_INDEX"]

    feature_before_germination: list[int] = spore_feature[0: spore_germination_frame]

    initial_feature = np.mean(feature_before_germination[0: 1]) #averaged over first and last frames
    INITIAL_FEATURE_COLUMN.append(initial_feature)
    
    last_feature = np.mean(feature_before_germination[-2: -1])
    LAST_FEATURE_COLUMN.append(last_feature)

    diff_feature = abs(last_feature - initial_feature)
    DIFF_FEATURE_COLUMN.append(diff_feature)

  df[f"INITIAL_{feature}"] = INITIAL_FEATURE_COLUMN
  df[f"LAST_{feature}"] = LAST_FEATURE_COLUMN
  df[f"DIFF_{feature}"] = DIFF_FEATURE_COLUMN


add_initial_last_measure_feature(df, "INTENSITY")
add_initial_last_measure_feature(df, "AREA")
add_initial_last_measure_feature(df, "ELLIPSE MINOR")
add_initial_last_measure_feature(df, "ELLIPSE MAJOR")
add_initial_last_measure_feature(df, "ELLIPSE ASPECT RATIO")
add_initial_last_measure_feature(df, "PERIMETER")
add_initial_last_measure_feature(df, "CIRCULARITY")

### PENDING: Swarmplot of germination frame and feature values

### Sample of feature behaviors over time

In [318]:
def plot_feature_example(feature_column_title, feature: str, frames: list[int]):
    FEATURE = df[feature_column_title].apply(ast.literal_eval)

    #plot initializing
    plt.clf()
    palette = sns.color_palette('Spectral', n_colors=len(frames))
    custom_handles = []
    spore_labels = []
    plt.figure(figsize=(12,8))

    #germinant intervals
    for frame in frames[0:5]:
      plt.axvline(x = frame - 1, color='silver', linestyle='--', linewidth = 2) 
    custom_handles.append(plt.Line2D([0], [0], color='silver', linestyle='dashed', label="Germinant Exposure", linewidth = 2))

    #---
    for frame in frames[:: -1]: 
      tmp = 0
      for spore_index, feature_row in enumerate(FEATURE):
        germination_frame = df["GERMINATION_INDEX"][spore_index]
        if germination_frame == frame:
          tmp += 1
          if tmp == 3:
            frame_index = frames.index(frame)
            colorline = palette[frame_index % len(palette)]
            #before germination
            sns.lineplot(x=range(germination_frame), y=feature_row[:germination_frame], linewidth=10, color = colorline, label = f"Spore {frame_index + 1}")#, label=f'Germinated at L-alanine Exposure {frames.index(frame) + 1}', )
            #after germination
            sns.lineplot(x = range(germination_frame-1, len(feature_row)), y=feature_row[germination_frame-1:], linestyle = '-', color = colorline, linewidth = 3)
            #legend for spore labels
            custom_handles.append(plt.Line2D([0], [0], color=colorline, lw=4, linestyle='None', marker='o', label=f"Spore {frame_index + 1}"))
            spore_labels.append((f"Spore {frame_index + 1}", colorline))  
            break  

    #axis fonts size
    axis_fontsize = 32
    #plt.xlabel('Frame', fontsize=axis_fontsize) 
    #plt.ylabel(f"{feature}", fontsize=axis_fontsize)  
   
    #axis ticks and limits
    plt.xticks(fontsize = 20)  
    plt.yticks(fontsize = 20)
   # plt.xlim(0, 95)
    plt.ylim(0, 1)

    #legend
    custom_handles = custom_handles[::-1]
    plt.legend(handles=custom_handles, loc='best', shadow = True, fontsize=20)  
    plt.tight_layout(rect=[0, 0, 0.85, 1])  

    plt.savefig(f"{plots_folder}sample_{feature}.jpg")
    plt.clf()

plot_feature_example("INTENSITY", "Electrochemical Potential", sample_germination_frames)
plot_feature_example("AREA", "Size", sample_germination_frames)
plot_feature_example("ELLIPSE MINOR", "Ellipse Minor", sample_germination_frames)
plot_feature_example("ELLIPSE MAJOR", "Ellipse Major", sample_germination_frames)
plot_feature_example("PERIMETER", "Perimeter", sample_germination_frames)
plot_feature_example("CIRCULARITY", "Circularity", sample_germination_frames)
plot_feature_example("ELLIPSE ASPECT RATIO", "Ellipse Aspect Ratio", sample_germination_frames)

<Figure size 640x480 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

### PENDING: Feature Trends

In [319]:
def feature_trends(df, feature: str):
    germination_groups = df.groupby("GERMINATION_INDEX")
    for germination_frame, group in germination_groups:
        FEATURE = group[feature].apply(ast.literal_eval)
        
        mean_features = []
        
        # Calculate the maximum length of the time series to pad shorter series
        max_length = max(FEATURE.apply(len))
        
        # Create an array to accumulate the sums
        sum_features = np.zeros(max_length)
        count_features = np.zeros(max_length)
        
        for row in FEATURE:
            row_len = len(row)
            sum_features[:row_len] += row
            count_features[:row_len] += 1
            plt.scatter(range(row_len), row, s = .5, color = "lightblue")
        
        mean_features = sum_features / count_features
        
        # Plot the mean feature trend
        plt.plot(range(0, germination_frame), mean_features[:germination_frame])
        plt.plot(range(germination_frame-1, len(mean_features)), mean_features[germination_frame -1:])
        
        plt.xlabel('Time Index')
        plt.ylabel(f'{feature} Value')
        #plt.title(f'{feature} Trends for Exposure {exposure_number}')
        plt.legend()
        plt.show()

#feature_trends(df, "INTENSITY")
        

### Heatmap of Spore Features over the Population

In [320]:
def plot_feature_allspores(df, feature_column_title, feature: str):
  plt.clf()
  exp_length = np.min(len(df["GERMINATION"][:]))
     

  df = df.sort_values(by='GERMINATION_INDEX')
  FEATURE_pre = df[feature_column_title].apply(ast.literal_eval).tolist()

  FEATURE = []
  for spore_row in FEATURE_pre:
     FEATURE.append(spore_row[:exp_length])
     
  Z = np.array(FEATURE)


  sns.heatmap(data = Z)#, cmap=plt.cm.viridis)
  for frame in germinant_exposures:
      plt.axvline(x=frame, color='grey', linestyle="dotted", linewidth = 2)
  plt.xlabel("Frame")
  plt.ylabel("Spore Index")
  plt.savefig(f"{plots_folder + feature}.jpg")
  plt.clf()

plot_feature_allspores(df, "INTENSITY", "Intensity")
plot_feature_allspores(df, "AREA", "Area")
plot_feature_allspores(df, "ELLIPSE MINOR", "Minor")
plot_feature_allspores(df, "ELLIPSE MAJOR", "Major")
plot_feature_allspores(df, "PERIMETER", "Perimeter")
plot_feature_allspores(df, "CIRCULARITY", "Circularity")
plot_feature_allspores(df, "ELLIPSE ASPECT RATIO", "AspectRatio")
plot_feature_allspores(df, "GERMINATION", "Germination")

<Figure size 640x480 with 0 Axes>

## Feature Relationships

### Heatmap showing Pearson Correlation Coefficient between Physiological Features

In [321]:
def feature_heatmap(df, features_list, plot_title, custom_labels, bold_labels):
  '''takes in df in format where each row denotes a spore and each column is a feature and each cell is feature over time in a list'''
  plt.clf()

  correlation_matrices = []

  for spore_index in range(len(df)):
    spore_features = []
    for feature in features_list:
      spore_feature = ast.literal_eval(df.loc[spore_index, feature])
      spore_features.append(spore_feature)
    spore_df = pd.DataFrame(spore_features).T
    spore_df.columns = features_list

    spore_feature_correlation = spore_df.corr()

    correlation_matrices.append(spore_feature_correlation)
  
  avg_correlation_matrix = np.mean(correlation_matrices, axis=0)
  avg_correlation_df = pd.DataFrame(avg_correlation_matrix, index=features_list, columns=features_list)
  avg_correlation_df.rename(columns=custom_labels, index=custom_labels, inplace=True)

  # Plot the heatmap for the average correlation matrix
  plt.figure(figsize = ((14,13)))
  ax = sns.heatmap(avg_correlation_df, annot=True, vmin=-1, vmax=1, cmap="coolwarm", 
                   xticklabels=True, yticklabels=True, annot_kws={"size": 15}, #numbers in heatmap
                   cbar_kws={'label': "$r_{F_1, F_2}$"}, cbar = False)

  ax.set_aspect('equal')

  # Customize tick labels
  tick_font_size = 24

  for tick in ax.get_xticklabels():
      if bold_labels and tick.get_text() in bold_labels:
          tick.set_weight('bold')
      tick.set_fontsize(tick_font_size) #labels size
  
  for tick in ax.get_yticklabels():
      if bold_labels and tick.get_text() in bold_labels:
          tick.set_weight('bold')
      tick.set_fontsize(tick_font_size)
  plt.yticks(rotation=0) 
  plt.xticks(rotation=90) 


      
  #padding between labels and heatmap
  ax.tick_params(axis='x', pad=10)
  ax.tick_params(axis='y', pad=10)

  #colorbar with the same height as the heatmap
  divider = make_axes_locatable(ax)
  cax = divider.append_axes("right", size="5%", pad=0.5)
  cbar = plt.colorbar(ax.collections[0], cax=cax)

  # Customize colorbar label size
  cbar.ax.tick_params(labelsize=20)
  cbar.ax.yaxis.label.set_size(30)
  cbar.ax.yaxis.label.set_text("Pearson Correlation Coefficient {$r_{F_1, F_2}$}")
  cbar.ax.yaxis.labelpad = 15

  #save
  plt.tight_layout()
  plt.savefig(f"{plots_folder + plot_title}.jpg")
  plt.clf()

  # feature heatmap call 

heatmap_labels = {
  'GERMINATION': 'Germination',
  'INTENSITY': 'Intensity',
  "AREA": "Area",
  "ELLIPSE MINOR": "Ellipse Minor",
  "ELLIPSE MAJOR": "Ellipse Major",
  "PERIMETER": "Perimeter",
  "CIRCULARITY": "Circularity",
  "GERMINANT EXPOSURE": "Germinant Exposure",
  "ELLIPSE ASPECT RATIO": "Ellipse Aspect Ratio"
  }
features = ["GERMINATION", "INTENSITY", "AREA", "ELLIPSE MINOR", "ELLIPSE MAJOR", "PERIMETER", "CIRCULARITY", "ELLIPSE ASPECT RATIO"]
bold_labels = ["Intensity", "Area", "Ellipse Minor", "Ellipse Major", "Perimeter", "Ellipse Aspect Ratio"]
feature_heatmap(df, features, "Feature Correlations over Time", heatmap_labels, bold_labels)

<Figure size 640x480 with 0 Axes>

<Figure size 1400x1300 with 0 Axes>